In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: 03
### Team Member Names: Sherry, Kelly, Ifan
### Team Strategy Chosen: SAFE

In [11]:
tickers_df = pd.read_csv('Tickers_Example.csv')
#When the CSV file is read, it takes the first ticker and makes it the name of the column, so we need to extract it
tickers_list = list(tickers_df.columns)
#Now add the rest of the tickers in after it:
tickers_list += tickers_df.iloc[:, 0].tolist()
tickers_num = len(tickers_list)

#Dates to pull closing data from, change as needed
start_date = '2020-01-01'
end_date = '2022-01-02'

# Dictionary of all the stocks
stocks_dict = {}
stocks_full_info = {}

# Function produces the daily closing prices of a stock
def get_closing_prices(ticker, start, end):
    stock = yf.Ticker(ticker)
    hist = stock.history(start=start, end=end) #Daily
    return hist.Close

for i in range (tickers_num):
    stocks_dict[tickers_list[i]] = get_closing_prices(tickers_list[i], start_date, end_date)
    stocks_full_info[tickers_list[i]] = yf.Ticker(tickers_list[i])

# Create DataFrame
all_prices = pd.DataFrame(stocks_dict)

tickers_list_temp = tickers_list.copy()
#Some tickers are delisted, so we drop those columns entirely
for i in range (tickers_num):
    if all_prices[tickers_list[i]].isnull().all():
        all_prices.drop(columns=[tickers_list[i]], inplace=True)
        tickers_list_temp.remove(tickers_list[i]) #Getting rid of the tickers from our list too to match

tickers_list = tickers_list_temp.copy()
tickers_num = len(tickers_list)

#Drop the rows at the bottom of the DataFrame where only a few stocks have values and where the dates are strange
all_prices = all_prices.dropna() 

display(tickers_list)

#Display the entire DataFrame, so that we can manually check to make sure the values are sensible
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(all_prices)

- AGN: No data found, symbol may be delisted
- CELG: No data found, symbol may be delisted
- PCLN: No data found for this date range, symbol may be delisted
- RTN: No data found, symbol may be delisted
- TWX: No data found for this date range, symbol may be delisted


['AAPL',
 'ABBV',
 'ABT',
 'ACN',
 'AIG',
 'AMZN',
 'AXP',
 'BA',
 'BAC',
 'BIIB',
 'BK',
 'BLK',
 'BMY',
 'C',
 'CAT',
 'CL',
 'CMCSA',
 'COF',
 'COP',
 'COST',
 'CSCO',
 'CVS',
 'GM',
 'GOOG',
 'JPM',
 'KMI',
 'KO',
 'LLY',
 'LMT',
 'MO',
 'MON',
 'MRK',
 'MS',
 'MSFT',
 'NEE',
 'NKE',
 'ORCL',
 'OXY',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RY.TO',
 'SBUX',
 'SLB',
 'SO',
 'SPG',
 'T',
 'TD.TO',
 'TGT',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB']

,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,BK,BLK,BMY,C,CAT,CL,CMCSA,COF,COP,COST,CSCO,CVS,GM,GOOG,JPM,KMI,KO,LLY,LMT,MO,MON,MRK,MS,MSFT,NEE,NKE,ORCL,OXY,PEP,PFE,PG,PM,PYPL,QCOM,RY.TO,SBUX,SLB,SO,SPG,T,TD.TO,TGT,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-03-16 00:00:00,124.265915,102.991287,114.960411,257.676666,45.678757,154.593002,140.325363,255.210007,36.480057,259.920013,43.974262,698.315796,58.655907,68.990318,217.884537,72.816872,55.796642,124.143257,52.917679,323.250336,47.165890,72.076767,56.989883,104.625999,146.212967,14.289574,48.974052,186.289185,332.890930,43.404175,9.785,69.581039,78.276802,234.083542,73.130455,142.769958,64.960350,27.963648,128.622879,33.759586,123.469925,80.504593,249.309998,129.892365,108.809334,107.442192,27.394434,57.139626,107.247581,19.613205,76.842148,174.705963,169.330261,347.556335,205.282028,155.261246,50.686958
2021-03-17 00:00:00,123.464317,97.601997,114.376961,258.410095,46.006889,156.786499,141.676712,263.589996,36.780502,264.149994,44.231026,703.289490,58.921234,69.244331,224.783005,72.759285,55.277504,127.196350,53.151501,325.166626,46.928493,71.059052,59.913212,104.554001,147.717865,14.558847,48.993176,182.219894,336.621368,43.702667,9.680,70.024933,79.528618,233.423767,70.782661,142.937759,64.348175,28.827276,128.248566,33.721893,122.971992,81.417351,251.470001,130.154785,109.203354,106.593010,27.638506,56.626534,108.208481,19.717846,76.935684,172.951218,171.442825,345.184418,205.863831,153.191986,51.298897
2021-03-18 00:00:00,119.278252,96.421913,114.853432,260.971954,46.151665,151.399506,139.620316,256.059998,37.739994,262.269989,44.411720,698.953247,59.063370,69.470123,221.944702,73.018448,53.710510,123.969337,49.924820,319.032562,46.339748,71.510300,59.134987,101.810997,150.156158,14.145961,48.352554,179.487518,343.746582,44.396225,9.770,69.997742,79.604492,227.200180,69.007278,141.309204,64.853462,26.425003,127.202370,33.703049,123.240112,82.658699,238.410004,126.102020,109.794334,103.852440,26.603649,56.346664,106.908966,19.626289,77.655937,174.124268,166.672836,354.858398,208.831055,152.762863,52.974651
2021-03-19 00:00:00,118.743858,96.096710,116.934448,257.930939,45.196205,153.748001,137.789124,255.820007,37.342628,267.070007,44.078869,704.660889,59.413982,68.689255,216.758820,71.972176,53.316357,122.152954,49.195305,324.890015,46.510674,70.262138,59.683735,102.160004,147.765503,14.289574,48.582035,179.838806,340.505280,44.378666,9.750,70.215157,78.656143,226.835831,68.725929,135.703018,64.387039,27.894163,129.093216,33.476917,122.579376,82.556938,241.279999,127.336319,109.325294,102.617241,26.408390,55.693642,106.789993,19.462791,77.094696,181.821762,168.221420,358.318237,204.409317,151.895111,51.976727
2021-03-22 00:00:00,122.108551,98.401093,118.120796,259.319458,44.617134,155.543503,137.044907,251.229996,36.499443,272.339996,42.994728,705.897095,60.058346,67.701393,217.461212,72.730484,54.191181,120.983887,48.858608,330.401825,47.764126,70.540565,57.967651,101.929497,143.793686,14.163911,48.763699,181.351364,342.989044,45.335590,9.700,70.215157,77.688812,232.389786,69.550560,136.472916,64.445335,27.258848,132.260544,33.919758,124.657326,83.232193,244.384995,130.319992,108.799957,103.804176,26.994160,55.684307,105.472160,19.613205,76.832802,182.296814,172.159760,359.572815,200.016632,153.582947,51.063534
2021-03-23 00:00:00,121.267387,97.416153,117.167816,260.131012,43.410748,156.875000,133.255234,241.250000,35.762863,267.209991,42.195892,692.038269,59.338169,66.713531,209.985413,74.631042,53.970081,118.268936,48.362907,336.180328,47.488750,69.033165,56.032074,102.648003,142.355499,14.074152,49.136593,179.243561,339.105133,44.343544,9.660,69.091858,75.033447,233.955536,71.811035,135.337845,65.368484,26.395224,133.853806,33.316742,126.974655,81.446930,243.770004,128.794144,108.668625,102.530388,26.037407,56.561230,101.042793,19.619747,76.439934,183.149902,170.897949,361.180237,201.742691,152.438660,50.442192
2021-03-24 00:

In [3]:
## doesnt work yet 
# Remove invalid stocks per assignment requirement (ones outside of the U.S & ones with average monthly volume of less than 200,000 shares)

def filter_market(df):
    df_transposed = df.T
    for i in range(len(df_transposed)):
        # Add country of market as a column to dataframe
        df_transposed["Market"].iloc[i] = stocks_full_info[i].info['market']
    
    # Drops the stocks in markets that are not in the U.S
    df_transposed.drop(df_transposed[df_transposed.Market != "us_market"].index, inplace=True)
    df_transposed = df_transposed.drop('Market', axis=1)
    return_df = df.T

    return return_df


def filter_volume(df):

    start = "2022-01-01"
    end = "2022-10-31"

    df_transposed = df.T

    for i in range(len(df_transposed)):
        ticker = yf.Ticker(df_transposed.iloc[i])
        ticker_hist = ticker.history(start=start, end=end)

        volume_hist = ticker_hist.loc[(ticker_hist.index >= pd.to_datetime(start)) & (ticker_hist.index <= pd.to_datetime(end))]
        df_transposed["Volume"].iloc[i] = (volume_hist.info['averageDailyVolume10Day'].mean())*3

    # Drops the stocks with volumes less than average monthly volume of 200,000
    df_transposed.drop(df_transposed[df_transposed.Volume < 200000].index, inplace=True)
    df_transposed = df_transposed.drop('Volume', axis=1)
    return_df = df.T

    return return_df

all_prices = all_prices.loc[filter_market(all_prices)]
all_prices = all_prices.loc[filter_volume(all_prices)]

# Drops stocks with more than specified number of NaN values
max_number_of_nans = 20
all_prices = all_prices.loc[:, (all_prices.isnull().sum(axis=0) <= max_number_of_nans)]


KeyError: 0

In [1]:
# Calculate BETA against S&P500
def stock_beta(df_column):

    index_ticker = yf.Ticker('SPY')
    index_prices = index_ticker.history(start=df_column.index[0], end=df_column.index[(len(df_column))])

    # Will need to double check this equation
    market_var= index_prices.var()
    beta = df_column.cov()/market_var
    
    return beta

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.